# Mental Health Risk Predictor

This notebook implements a Mental Health Risk Predictor using Artificial Neural Networks (ANN). The model analyzes lifestyle data, digital behavior, sleep patterns, and survey responses to predict mental health risks.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Data Loading and Exploration

In this section, we'll load and explore the dataset from various sources including app usage, sleep patterns, and survey responses.

In [ ]:
# Function to load and process data
def load_app_usage_data():
    app_usage_df = pd.read_csv('E:/dataset/app_usage/app_usage_stats.csv')
    return app_usage_df

def load_sensing_data():
    sensing_df = pd.read_csv('E:/dataset/sensing/sensing_data.csv')
    return sensing_df

def load_survey_data():
    survey_df = pd.read_csv('E:/dataset/survey/survey_responses.csv')
    return survey_df

# Load the datasets
try:
    app_usage_df = load_app_usage_data()
    sensing_df = load_sensing_data()
    survey_df = load_survey_data()
    
    print("Data loaded successfully!")
    print(f"App usage data shape: {app_usage_df.shape}")
    print(f"Sensing data shape: {sensing_df.shape}")
    print(f"Survey data shape: {survey_df.shape}")
except Exception as e:
    print(f"Error loading data: {str(e)}")

## Data Preprocessing

Preprocess the data by handling missing values, encoding categorical variables, and scaling numerical features.

In [ ]:
def preprocess_data(app_usage_df, sensing_df, survey_df):
    # Merge datasets on user_id and timestamp
    merged_df = pd.merge(app_usage_df, sensing_df, on=['user_id', 'timestamp'], how='outer')
    merged_df = pd.merge(merged_df, survey_df, on=['user_id', 'timestamp'], how='outer')
    
    # Handle missing values
    merged_df = merged_df.fillna(merged_df.mean(numeric_only=True))
    
    # Extract features
    features = [
        'screen_time',
        'app_switches',
        'social_media_time',
        'sleep_duration',
        'physical_activity',
        'stress_level',
        'anxiety_score',
        'depression_score'
    ]
    
    # Prepare features and target
    X = merged_df[features]
    y = merged_df['mental_health_risk']  # Assuming this is our target variable
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, scaler

# Preprocess the data
try:
    X_scaled, y, scaler = preprocess_data(app_usage_df, sensing_df, survey_df)
    print("Data preprocessing completed successfully!")
    print(f"Feature matrix shape: {X_scaled.shape}")
    print(f"Target vector shape: {y.shape}")
except Exception as e:
    print(f"Error in preprocessing: {str(e)}")

## Model Building

Build and train the ANN model for mental health risk prediction.

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the model
def create_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Create and compile the model
model = create_model(X_train.shape[1])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping],
                    verbose=1)

## Model Evaluation

Evaluate the model's performance using various metrics.

In [ ]:
# Evaluate the model
def evaluate_model(model, X_test, y_test):
    # Make predictions
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Evaluate the model
evaluate_model(model, X_test, y_test)

## Save the Model

Save the trained model and preprocessing objects for future use.

In [ ]:
# Save the model and scaler
import joblib

# Save the model
model.save('E:/Deep learning/ANN/Mental_Health_Predictor/models/mental_health_predictor.h5')

# Save the scaler
joblib.dump(scaler, 'E:/Deep learning/ANN/Mental_Health_Predictor/models/scaler.joblib')

print("Model and scaler saved successfully!")